<a href="https://colab.research.google.com/github/NarayaniSuresh/ADVANCEDDL/blob/main/attention_with_lstm_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SINGLE HEAD AND MULTI HEAD ATTENTION**

In [ ]:
#importing necessary libraries
import tensorflow as tf

from tensorflow import keras
from tensorflow.python.keras import Input
from tensorflow.python.keras.layers import Concatenate



In [ ]:
vocab_size=10000
pad_id=0
start_id=1
oov_id=2
index_offset=2

In [ ]:
(x_train,y_train),(x_test,y_test)=keras.datasets.imdb.load_data(num_words=vocab_size,start_char=start_id,oov_char=oov_id,index_from=index_offset)

17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
word2idx=tf.keras.datasets.imdb.get_word_index()

1641221/1641221 [==============================] - 0s 0us/step


In [ ]:
idx2word={v+index_offset: k for k, v in word2idx.items()}
idx2word[pad_id]='<PAD>'
idx2word[start_id]='<START>'
idx2word[oov_id]='<OOV>'

In [ ]:
max_len=200
rnn_cell_size=128

In [ ]:
from tensorflow.keras.preprocessing import sequence
x_train=sequence.pad_sequences(x_train,maxlen=max_len,padding='post',truncating='post',value=pad_id)
x_test=sequence.pad_sequences(x_test,maxlen=max_len,padding='post',truncating='post',value=pad_id)

In [ ]:
!pip install keras_self_attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras_self_attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18894 sha256=cc1987e7fcf26ade155673ce18d9ec3597399813991c91aa1db3dff8b907429f
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras_self_attention


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Flatten
from keras_self_attention import SeqSelfAttention



model=Sequential()
model.add(Embedding(vocab_size,100,input_length=max_len ))
model.add(Bidirectional(LSTM(units=16, return_sequences=True, dropout=0.5, recurrent_dropout=0.7)))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          1000000   
                                                                 
 bidirectional (Bidirection  (None, 200, 32)           14976     
 al)                                                             
                                                                 
 seq_self_attention (SeqSel  (None, 200, 32)           2113      
 fAttention)                                                     
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 1)                 6401      
                                                                 
Total params: 1023490 (3.90 MB)
Trainable params: 102349

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=2,batch_size=128,validation_split=0.2,verbose=1)

Epoch 1/2
157/157 [==============================] - 478s 3s/step - loss: 0.4238 - accuracy: 0.7974 - val_loss: 0.3266 - val_accuracy: 0.8646
Epoch 2/2
157/157 [==============================] - 465s 3s/step - loss: 0.2453 - accuracy: 0.9025 - val_loss: 0.3255 - val_accuracy: 0.8678


In [ ]:
results=model.evaluate(x_test,y_test)
`
print(results)

782/782 [==============================] - 239s 305ms/step - loss: 0.3554 - accuracy: 0.8494
[0.35541969537734985, 0.8493599891662598]


MULTI HEAD ATTENTION

In [ ]:


!pip install keras-multi-head

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Flatten
from keras_multi_head import MultiHead

  Preparing metadata (setup.py) ... done
  Created wheel for keras-multi-head: filename=keras_multi_head-0.29.0-py3-none-any.whl size=14976 sha256=e540d889545539fc3542475eb0e07acd1220fbdffd4b42ef7b394fd6e6760dac
  Stored in directory: /root/.cache/pip/wheels/cb/23/4b/06d7ae21714f70fcc25b48f972cc8e5e7f4b6b764a038b509d
Successfully built keras-multi-head


In [ ]:
model=Sequential()
model.add(Embedding(vocab_size,100,input_length=max_len ))
model.add(MultiHead(LSTM(units=32),layer_num=5,name="Multi-LSTMS"))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.build()
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 200, 100)          1000000   
                                                                 
 Multi-LSTMS (MultiHead)     (None, 32, 5)             85120     
                                                                 
 flatten_2 (Flatten)         (None, 160)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 161       
                                                                 
Total params: 1085281 (4.14 MB)
Trainable params: 1085281 (4.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(x_train,y_train,epochs=2,batch_size=128,validation_split=0.2,verbose=1)

Epoch 1/2
157/157 [==============================] - 191s 1s/step - loss: 0.6692 - accuracy: 0.5695 - val_loss: 0.5784 - val_accuracy: 0.7038
Epoch 2/2
157/157 [==============================] - 181s 1s/step - loss: 0.5428 - accuracy: 0.7355 - val_loss: 0.5629 - val_accuracy: 0.7404


In [ ]:
results=model.evaluate(x_test,y_test)

print(results)

782/782 [==============================] - 72s 93ms/step - loss: 0.5703 - accuracy: 0.7338
[0.5702508687973022, 0.7338399887084961]
